<a href="https://colab.research.google.com/github/kunalrdeshmukh/End-to-end-compression/blob/master/End_to_end_image_compression_stl10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Image Compression framework :

[End to end image compression framework ](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7999241)

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

import os

import torch
import torchvision
from torch import nn , optim
from torchvision import datasets, transforms
from torch.autograd import Variable

!pip install Pillow==4.0.0
!pip install PIL
!pip install image
from PIL import Image

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:
import math
irange = range


def make_grid(tensor, nrow=8, padding=2,
              normalize=False, range=None, scale_each=False, pad_value=0):
    """Make a grid of images.
    Args:
        tensor (Tensor or list): 4D mini-batch Tensor of shape (B x C x H x W)
            or a list of images all of the same size.
        nrow (int, optional): Number of images displayed in each row of the grid.
            The Final grid size is (B / nrow, nrow). Default is 8.
        padding (int, optional): amount of padding. Default is 2.
        normalize (bool, optional): If True, shift the image to the range (0, 1),
            by subtracting the minimum and dividing by the maximum pixel value.
        range (tuple, optional): tuple (min, max) where min and max are numbers,
            then these numbers are used to normalize the image. By default, min and max
            are computed from the tensor.
        scale_each (bool, optional): If True, scale each image in the batch of
            images separately rather than the (min, max) over all images.
        pad_value (float, optional): Value for the padded pixels.
    Example:
        See this notebook `here <https://gist.github.com/anonymous/bf16430f7750c023141c562f3e9f2a91>`_
    """
    if not (torch.is_tensor(tensor) or
            (isinstance(tensor, list) and all(torch.is_tensor(t) for t in tensor))):
        raise TypeError('tensor or list of tensors expected, got {}'.format(type(tensor)))

    # if list of tensors, convert to a 4D mini-batch Tensor
    if isinstance(tensor, list):
        tensor = torch.stack(tensor, dim=0)

    if tensor.dim() == 2:  # single image H x W
        tensor = tensor.view(1, tensor.size(0), tensor.size(1))
    if tensor.dim() == 3:  # single image
        if tensor.size(0) == 1:  # if single-channel, convert to 3-channel
            tensor = torch.cat((tensor, tensor, tensor), 0)
        tensor = tensor.view(1, tensor.size(0), tensor.size(1), tensor.size(2))

    if tensor.dim() == 4 and tensor.size(1) == 1:  # single-channel images
        tensor = torch.cat((tensor, tensor, tensor), 1)

    if normalize is True:
        tensor = tensor.clone()  # avoid modifying tensor in-place
        if range is not None:
            assert isinstance(range, tuple), \
                "range has to be a tuple (min, max) if specified. min and max are numbers"

        def norm_ip(img, min, max):
            img.clamp_(min=min, max=max)
            img.add_(-min).div_(max - min + 1e-5)

        def norm_range(t, range):
            if range is not None:
                norm_ip(t, range[0], range[1])
            else:
                norm_ip(t, float(t.min()), float(t.max()))

        if scale_each is True:
            for t in tensor:  # loop over mini-batch dimension
                norm_range(t, range)
        else:
            norm_range(tensor, range)

    if tensor.size(0) == 1:
        return tensor.squeeze()

    # make the mini-batch of images into a grid
    nmaps = tensor.size(0)
    xmaps = min(nrow, nmaps)
    ymaps = int(math.ceil(float(nmaps) / xmaps))
    height, width = int(tensor.size(2) + padding), int(tensor.size(3) + padding)
    grid = tensor.new(3, height * ymaps + padding, width * xmaps + padding).fill_(pad_value)
    k = 0
    for y in irange(ymaps):
        for x in irange(xmaps):
            if k >= nmaps:
                break
            grid.narrow(1, y * height + padding, height - padding)\
                .narrow(2, x * width + padding, width - padding)\
                .copy_(tensor[k])
            k = k + 1
    return grid


def save_image(tensor, filename, nrow=8, padding=2,
               normalize=False, range=None, scale_each=False, pad_value=0):
    """Save a given Tensor into an image file.
    Args:
        tensor (Tensor or list): Image to be saved. If given a mini-batch tensor,
            saves the tensor as a grid of images by calling ``make_grid``.
        **kwargs: Other arguments are documented in ``make_grid``.
    """
    from PIL import Image
    grid = make_grid(tensor, nrow=nrow, padding=padding, pad_value=pad_value,
                     normalize=normalize, range=range, scale_each=scale_each)
    ndarr = grid.mul(255).clamp(0, 255).byte().permute(1, 2, 0).cpu().numpy()
    im = Image.fromarray(ndarr)
    im.save(filename)

In [0]:
img_transform = transforms.Compose([
    transforms.ToTensor()
])

trainset = datasets.STL10(root='./data', split='train',download=True, transform=img_transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=16,
                                          shuffle=True, num_workers=2)

testset = datasets.STL10(root='./data', split='test',download=True, transform=img_transform)

test_loader = torch.utils.data.DataLoader(testset, batch_size=16,shuffle=False, num_workers=2)

Files already downloaded and verified


In [0]:
# code block to text image loader
# print(trainloader)

### Parameters

In [0]:
CHANNELS = 3
HEIGHT = 96
WIDTH = 96
EPOCHS = 20
LOG_INTERVAL = 50

### Define the network

In [0]:
class Interpolate(nn.Module):
    def __init__(self, size, mode):
        super(Interpolate, self).__init__()
        self.interp = nn.functional.interpolate
        self.size = size
        self.mode = mode
        
    def forward(self, x):
        x = self.interp(x, size=self.size, mode=self.mode, align_corners=False)
        return x

In [0]:
class End_to_end(nn.Module):
  def __init__(self):
    super(End_to_end, self).__init__()
    
    # Encoder
    # TODO : try with padding = 0
    self.conv1 = nn.Conv2d(CHANNELS, 64, kernel_size=3, stride=1, padding=1)
    self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=2, padding=0)
    self.bn1 = nn.BatchNorm2d(64, affine=False)
    self.conv3 = nn.Conv2d(64, CHANNELS, kernel_size=3, stride=1, padding=1)
    
    # Decoder
    #TODO : try ConvTranspose2d
    self.interpolate = Interpolate(size=HEIGHT, mode='bilinear')
    self.deconv1 = nn.Conv2d(CHANNELS, 64, 3, stride=1, padding=1)
    self.deconv2 = nn.Conv2d(64, 64, 3, stride=1, padding=1)
    self.bn2 = nn.BatchNorm2d(64, affine=False)
    
    self.deconv_n = nn.Conv2d(64, 64, 3, stride=1, padding=1)
    self.bn_n = nn.BatchNorm2d(64, affine=False)

    
    self.deconv3 = nn.ConvTranspose2d(64, CHANNELS, 3, stride=1, padding=1)
    
    self.relu = nn.ReLU()
  
  def encode(self, x):
    out = self.relu(self.conv1(x))
    out = self.relu(self.conv2(out))
    out = self.bn1(out)
    return self.conv3(out)
    
  
  def reparameterize(self, mu, logvar):
    pass
  
  def decode(self, z):
    upscaled_image = self.interpolate(z)
    out = self.relu(self.deconv1(upscaled_image))
    out = self.relu(self.deconv2(out))
    out = self.bn2(out)
    for _ in range(5):
      out = self.relu(self.deconv_n(out))
      out = self.bn_n(out)
    out = self.deconv3(out)
    final = upscaled_image + out
    return final,out,upscaled_image

    
  def forward(self, x):
    com_img = self.encode(x)
    final,out,upscaled_image = self.decode(com_img)
    return final, out, upscaled_image, com_img, x

  


In [0]:
CUDA = torch.cuda.is_available()
if CUDA:
  model = End_to_end().cuda()
else :
  model = end_to_end()
  
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [0]:
def loss_function(final_img,residual_img,upscaled_img,com_img,orig_img):

  com_loss = nn.MSELoss(size_average=False)(orig_img, final_img)
  rec_loss = nn.MSELoss(size_average=False)(residual_img,orig_img-upscaled_img)
  
  return com_loss + rec_loss

In [0]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data)
        optimizer.zero_grad()
        if CUDA:
          final, residual_img, upscaled_image, com_img, orig_im = model(data.cuda())
        else :
          final, residual_img, upscaled_image, com_img, orig_im = model(data)
        loss = loss_function(final, residual_img, upscaled_image, com_img, orig_im)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
        if batch_idx % LOG_INTERVAL == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data[0] / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


In [0]:
def test(epoch):
  
  model.eval()
  test_loss = 0
  for i, (data, _) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        if CUDA:
          final, residual_img, upscaled_image, com_img, orig_im = model(data.cuda())
        else :
          final, residual_img, upscaled_image, com_img, orig_im = model(data)
        test_loss += loss_function(final, residual_img, upscaled_image, com_img, orig_im).data[0]
        if epoch == EPOCHS and i == 0:
#             save_image(final.data[0],'reconstruction_final',nrow=8)
#             save_image(com_img.data[0],'com_img',nrow=8)
            n = min(data.size(0), 6)
            print("saving the image "+str(n))
            comparison = torch.cat([data[:n],
              final[:n].cpu()])


  test_loss /= len(test_loader.dataset)
  print('====> Test set loss: {:.4f}'.format(test_loss))

In [0]:
for epoch in range(1, EPOCHS+1):
    train(epoch)
    test(epoch)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/5000 (0%)]	Loss: 38111.566406
Train Epoch: 1 [800/5000 (16%)]	Loss: 5848.400879
Train Epoch: 1 [1600/5000 (32%)]	Loss: 1606.412842
Train Epoch: 1 [2400/5000 (48%)]	Loss: 756.815430
Train Epoch: 1 [3200/5000 (64%)]	Loss: 565.094238
Train Epoch: 1 [4000/5000 (80%)]	Loss: 462.755920
Train Epoch: 1 [4800/5000 (96%)]	Loss: 612.051575
====> Epoch: 1 Average loss: 2709.7944


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  # This is added back by InteractiveShellApp.init_path()


====> Test set loss: 497.4062
Train Epoch: 2 [0/5000 (0%)]	Loss: 843.137268
Train Epoch: 2 [800/5000 (16%)]	Loss: 535.899658
Train Epoch: 2 [1600/5000 (32%)]	Loss: 485.820160
Train Epoch: 2 [2400/5000 (48%)]	Loss: 450.467041
Train Epoch: 2 [3200/5000 (64%)]	Loss: 456.669586
Train Epoch: 2 [4000/5000 (80%)]	Loss: 288.428192
Train Epoch: 2 [4800/5000 (96%)]	Loss: 351.939423
====> Epoch: 2 Average loss: 428.2243
====> Test set loss: 442.4455
Train Epoch: 3 [0/5000 (0%)]	Loss: 365.625946
Train Epoch: 3 [800/5000 (16%)]	Loss: 382.670868
Train Epoch: 3 [1600/5000 (32%)]	Loss: 264.669983
Train Epoch: 3 [2400/5000 (48%)]	Loss: 651.008179
Train Epoch: 3 [3200/5000 (64%)]	Loss: 353.817596
Train Epoch: 3 [4000/5000 (80%)]	Loss: 477.154694
Train Epoch: 3 [4800/5000 (96%)]	Loss: 353.566467
====> Epoch: 3 Average loss: 383.3647
====> Test set loss: 599.5198
Train Epoch: 4 [0/5000 (0%)]	Loss: 406.398987
Train Epoch: 4 [800/5000 (16%)]	Loss: 401.010864
Train Epoch: 4 [1600/5000 (32%)]	Loss: 381.764343

Reference : https://github.com/L1aoXingyu/pytorch-beginner/tree/master/08-AutoEncoder

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive') 
torch.save(model.state_dict(), './net.pth')

In [0]:
model.load_state_dict(torch.load('net.pth'))

def save_images():
  epoch = EPOCHS
  model.eval()
  test_loss = 0
  for i, (data, _) in enumerate(test_loader):
        data = Variable(data, volatile=True)
        if CUDA:
          final, residual_img, upscaled_image, com_img, orig_im = model(data.cuda())
        else:
          final, residual_img, upscaled_image, com_img, orig_im = model(data)

        test_loss += loss_function(final, residual_img, upscaled_image, com_img, orig_im).data[0]
        if True:
#             save_image(final.data[0],'reconstruction_final',nrow=8)
#             save_image(com_img.data[0],'com_img',nrow=8)
            n = min(data.size(0), 6)
            print("saving the image "+str(n))
            comparison = torch.cat([data[:n],
              final[:n].cpu()])
            comparison = comparison.cpu()
#             print(comparison.data)
            save_image(com_img[i].data,
                         'compressed_' + str(i) +'.png', nrow=n)
            save_image(final[i].data,
                        'final_' + str(i) +'.png', nrow=n)
            save_image(orig_im[i].data,
                        'original_' + str(i) +'.png', nrow=n)


  test_loss /= len(test_loader.dataset)
  print('====> Test set loss: {:.4f}'.format(test_loss))

save_images()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  


saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6
saving the image 6


Exception ignored in: <bound method _DataLoaderIter.__del__ of <torch.utils.data.dataloader._DataLoaderIter object at 0x7f930930b470>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 399, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 378, in _shutdown_workers
    self.worker_result_queue.get()
  File "/usr/lib/python3.6/multiprocessing/queues.py", line 337, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.6/dist-packages/torch/multiprocessing/reductions.py", line 151, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python3.6/multiprocessing/resource_sharer.py", line 87, in get_connection
    c = Client(address, authkey=process.current_process().authkey)
  Fil

IndexError: ignored

In [0]:
train(0)

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 0 [0/5000 (0%)]	Loss: 254.253082
Train Epoch: 0 [800/5000 (16%)]	Loss: 264.752197
Train Epoch: 0 [1600/5000 (32%)]	Loss: 276.037109
Train Epoch: 0 [2400/5000 (48%)]	Loss: 273.387512
Train Epoch: 0 [3200/5000 (64%)]	Loss: 225.453827
Train Epoch: 0 [4000/5000 (80%)]	Loss: 353.847778
Train Epoch: 0 [4800/5000 (96%)]	Loss: 258.894104
====> Epoch: 0 Average loss: 272.8875
